#  CrimeLens — Video to 3D Gaussian Splat Model

Convert a short video into a `.ply` Gaussian Splat model viewable in the Unity VR Viewer.

**Prerequisites**: Runtime → Change runtime type → **GPU (G4 is preffered).**

Run cells **top to bottom**. Each section has its own Gradio UI for parameters.

In [1]:
# ── 0. Verify GPU ─────────────────────────────────────────────────────────
import subprocess, sys

result = subprocess.run(['nvidia-smi', '--query-gpu=name,memory.total', '--format=csv,noheader'],
                        capture_output=True, text=True)
if result.returncode != 0:
    print('⚠  No NVIDIA GPU detected!  Runtime → Change runtime type → T4 GPU')
    sys.exit('Please enable GPU and re-run.')
else:
    print('✓ GPU:', result.stdout.strip())

import torch
print('✓ CUDA available:', torch.cuda.is_available())
print('✓ PyTorch:', torch.__version__)
print('✓ Python:', sys.version.split()[0])

✓ GPU: NVIDIA RTX PRO 6000 Blackwell Server Edition, 97887 MiB
✓ CUDA available: True
✓ PyTorch: 2.10.0+cu128
✓ Python: 3.12.12


## Section 1 — Environment Setup
Installs COLMAP, clones gaussian-splatting, builds CUDA extensions. **~5 min** first run.

In [2]:
# ── 1. Install system & Python dependencies ───────────────────────────────
import os, subprocess, sys
from pathlib import Path

WORKSPACE = Path('/content/vid2splat')
WORKSPACE.mkdir(exist_ok=True)
GS_REPO   = Path('/content/gaussian-splatting')

def sh(cmd, **kwargs):
    """Run a shell command and stream output."""
    print(f'$ {cmd}')
    r = subprocess.run(cmd, shell=True, text=True, **kwargs)
    if r.returncode != 0:
        raise RuntimeError(f'Command failed (exit {r.returncode}): {cmd}')
    return r

# ── COLMAP ────────────────────────────────────────────────────────────────
print('Installing COLMAP...')
sh('apt-get install -qq -y colmap')
r = subprocess.run(['colmap', '--version'], capture_output=True, text=True)
print('✓ COLMAP:', r.stdout.strip() or r.stderr.strip())

# ── Xvfb (virtual display — needed for COLMAP GPU/OpenGL on headless Colab) ──
print('Installing Xvfb...')
sh('apt-get install -qq -y xvfb')
print('✓ Xvfb installed')

# ── FFmpeg (usually pre-installed) ───────────────────────────────────────
r = subprocess.run(['ffmpeg', '-version'], capture_output=True, text=True)
print('✓ FFmpeg:', r.stdout.split('\n')[0])

# ── Gaussian Splatting repo ───────────────────────────────────────────────
if not GS_REPO.exists():
    print('Cloning gaussian-splatting...')
    sh('git clone --recursive https://github.com/graphdeco-inria/gaussian-splatting /content/gaussian-splatting')
else:
    print('✓ gaussian-splatting already cloned')

# ── Build CUDA extensions ─────────────────────────────────────────────────
print('Building diff-gaussian-rasterization (CUDA extension)...')
sh('pip install -q /content/gaussian-splatting/submodules/diff-gaussian-rasterization')
print('Building simple-knn...')
sh('pip install -q /content/gaussian-splatting/submodules/simple-knn')

# ── Python packages ───────────────────────────────────────────────────────
print('Installing Python packages...')
sh('pip install -q gradio plotly plyfile tqdm imageio imageio-ffmpeg rich lpips')

print('\n✅ Environment ready!')


Installing COLMAP...
$ apt-get install -qq -y colmap
✓ COLMAP: ERROR: Command `--version` not recognized. To list the available commands, run `colmap help`.
Installing Xvfb...
$ apt-get install -qq -y xvfb
✓ Xvfb installed
✓ FFmpeg: ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
Cloning gaussian-splatting...
$ git clone --recursive https://github.com/graphdeco-inria/gaussian-splatting /content/gaussian-splatting
Building diff-gaussian-rasterization (CUDA extension)...
$ pip install -q /content/gaussian-splatting/submodules/diff-gaussian-rasterization
Building simple-knn...
$ pip install -q /content/gaussian-splatting/submodules/simple-knn
Installing Python packages...
$ pip install -q gradio plotly plyfile tqdm imageio imageio-ffmpeg rich lpips

✅ Environment ready!


## Section 2 — Video Upload & Project Setup

In [5]:
# ── 2. Upload video or mount Google Drive ────────────────────────────────
import gradio as gr
import shutil
from pathlib import Path

WORKSPACE = Path('/content/vid2splat')
state = {}  # shared pipeline state

def setup_project(video_file, project_name, use_drive, drive_path):
    if not project_name.strip():
        return '⚠ Enter a project name.', gr.update()

    proj_dir = WORKSPACE / project_name.strip().replace(' ', '_')
    input_dir = proj_dir / 'input'
    input_dir.mkdir(parents=True, exist_ok=True)

    if use_drive and drive_path.strip():
        # Mount Google Drive
        from google.colab import drive
        drive.mount('/content/drive')
        src = Path(drive_path.strip())
        if not src.exists():
            return f'⚠ Drive path not found: {src}', gr.update()
        video_dest = proj_dir / src.name
        shutil.copy2(src, video_dest)
        state['video_path'] = str(video_dest)
    elif video_file is not None:
        video_dest = proj_dir / Path(video_file.name).name
        shutil.copy2(video_file.name, video_dest)
        state['video_path'] = str(video_dest)
    else:
        return '⚠ Upload a video or provide a Drive path.', gr.update()

    state['project_dir'] = str(proj_dir)
    state['project_name'] = project_name.strip()
    sz = Path(state['video_path']).stat().st_size / 1024**2
    return f'✅ Project "{project_name}" ready.  Video: {Path(state["video_path"]).name} ({sz:.1f} MB)', gr.update(value=str(proj_dir))

with gr.Blocks(title='Vid2Splat — Upload') as upload_ui:
    gr.Markdown('### 📂 Upload Video')
    with gr.Row():
        proj_name  = gr.Textbox(label='Project name', placeholder='my_scene', value='my_scene')
        proj_dir_display = gr.Textbox(label='Project directory', interactive=False)
    video_input = gr.File(label='Upload video (.mp4 / .mov / .avi / .mkv)', file_types=['video'])
    with gr.Accordion('Google Drive (large files)', open=False):
        use_drive  = gr.Checkbox(label='Use Google Drive instead', value=False)
        drive_path = gr.Textbox(label='Drive path', placeholder='/content/drive/MyDrive/video.mp4')
    setup_btn  = gr.Button('Set up project', variant='primary')
    status_out = gr.Markdown()
    setup_btn.click(setup_project,
                    inputs=[video_input, proj_name, use_drive, drive_path],
                    outputs=[status_out, proj_dir_display])

upload_ui.launch(debug=False, share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://315f2804e217dd57f2.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


## Section 3 — Frame Extraction

In [6]:
# ── 3. Extract frames with FFmpeg ─────────────────────────────────────────
import gradio as gr
import subprocess, json
from pathlib import Path
import PIL.Image

PRESET_FRAME_TARGETS = {'Quick Preview': 60, 'Balanced': 120, 'High Quality': 250}


def get_video_info(video_path):
    cmd = ['ffprobe', '-v', 'quiet', '-print_format', 'json', '-show_streams', video_path]
    r = subprocess.run(cmd, capture_output=True, text=True)
    data = json.loads(r.stdout)
    for s in data.get('streams', []):
        if s.get('codec_type') == 'video':
            dur = float(s.get('duration', 0) or data.get('format', {}).get('duration', 0))
            num, den = map(int, s.get('r_frame_rate', '30/1').split('/'))
            return dur, (num / den if den else 30.0)
    return 0.0, 30.0


def extract_frames(preset, target_frames, max_frames, scale_pct):
    video_path = state.get('video_path')
    proj_dir   = Path(state.get('project_dir', '/content/vid2splat/scene'))
    if not video_path or not Path(video_path).exists():
        return '⚠ Run Section 2 first.', []

    input_dir = proj_dir / 'input'
    input_dir.mkdir(parents=True, exist_ok=True)

    # Clear old frames
    for f in input_dir.glob('*.jpg'):
        f.unlink()

    if preset != 'Custom':
        target_frames = PRESET_FRAME_TARGETS[preset]

    target_frames = min(int(target_frames), int(max_frames))
    scale = scale_pct / 100.0

    duration, native_fps = get_video_info(video_path)
    fps = min(target_frames / max(duration, 1), native_fps)

    vf = f'fps={fps:.4f}'
    if scale < 1.0:
        vf += f',scale=iw*{scale:.2f}:ih*{scale:.2f}'

    cmd = ['ffmpeg', '-y', '-i', video_path,
           '-qscale:v', '1', '-qmin', '1',
           '-vf', vf,
           str(input_dir / '%04d.jpg')]
    r = subprocess.run(cmd, capture_output=True, text=True)

    frames = sorted(input_dir.glob('*.jpg'))
    n = len(frames)
    state['n_frames'] = n
    state['input_dir'] = str(input_dir)

    if n == 0:
        return f'⚠ FFmpeg error:\n{r.stderr[-1000:]}', []

    # Sample thumbnails (every Nth)
    step = max(1, n // 8)
    thumbs = [str(f) for f in frames[::step]][:8]

    msg = (f'✅ Extracted **{n} frames** at {fps:.2f} fps  '
           f'| source duration {duration:.1f}s  '
           f'| est. COLMAP time ~{max(1, n // 60)} min')
    if n < 30:
        msg += '\n⚠ Very few frames — quality may be low.'
    return msg, thumbs


with gr.Blocks(title='Vid2Splat — Frames') as frames_ui:
    gr.Markdown('### 🎞 Frame Extraction')
    with gr.Row():
        preset_dd  = gr.Dropdown(['Quick Preview', 'Balanced', 'High Quality', 'Custom'],
                                  label='Preset', value='Balanced')
        target_sl  = gr.Slider(20, 9000, value=120, step=10, label='Target frames (Custom only)')
        max_sl     = gr.Slider(50, 9000, value=150, step=10, label='Max frames')
        scale_sl   = gr.Slider(25, 100, value=100, step=25, label='Image scale %')
    extract_btn = gr.Button('Extract frames', variant='primary')
    status_out  = gr.Markdown()
    gallery     = gr.Gallery(label='Sample frames', columns=4, height='auto')
    extract_btn.click(extract_frames,
                      inputs=[preset_dd, target_sl, max_sl, scale_sl],
                      outputs=[status_out, gallery])

frames_ui.launch(debug=False, share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://0cda697add6d672c1c.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


## Section 4 — COLMAP Structure-from-Motion

In [8]:

# ── 4. COLMAP SfM ─────────────────────────────────────────────────────────
import gradio as gr
import subprocess, struct, os, sys, time, traceback, tempfile
from pathlib import Path

# Per-step timeouts (seconds)
TIMEOUT_EXTRACT = 3600   # 60 min
TIMEOUT_MATCH   = 7200   # 120 min  — slowest step
TIMEOUT_MAPPER  = 3600   # 60 min


def count_registered_images(sparse_0: Path) -> int:
    images_bin = sparse_0 / 'images.bin'
    if images_bin.exists():
        with open(images_bin, 'rb') as f:
            data = f.read(8)
        if len(data) == 8:
            return struct.unpack('<Q', data)[0]
    images_txt = sparse_0 / 'images.txt'
    if images_txt.exists():
        with open(images_txt) as f:
            lines = [l for l in f if l.strip() and not l.startswith('#')]
        return len(lines) // 2
    return 0


def run_colmap(preset, matcher, overlap, use_gpu_chk):
    """
    Generator — yields live markdown status as COLMAP runs.
    Gradio 4.x streams yielded values directly to the UI component.
    """
    xvfb_proc  = None
    all_log    = []
    step_log   = []
    t0         = time.time()
    _n_frames  = 0
    _matcher   = matcher

    def elapsed():
        return '{:.0f}s'.format(time.time() - t0)

    def tail(n=30):
        lines = step_log[-n:] if step_log else []
        return '\n'.join(lines) if lines else '(waiting…)'

    def render(emoji, step_name, note=''):
        return (
            '**COLMAP** | {elapsed} elapsed | {n} images | GPU={gpu} | matcher={m}\n\n'
            '---\n'
            '{e} **{step}**{note}\n'
            '```\n{log}\n```'
        ).format(
            elapsed=elapsed(), n=_n_frames, gpu=use_gpu_chk,
            m=_matcher, e=emoji, step=step_name,
            note=' — ' + note if note else '',
            log=tail(),
        )

    yield '⏳ Initializing COLMAP…'

    try:
        _state = sys.modules['__main__'].__dict__.get('state') or globals().get('state')
        if _state is None:
            yield '❌ `state` not found — re-run Section 2 first.'
            return

        proj_dir_str = _state.get('project_dir')
        if not proj_dir_str:
            yield '❌ No project directory — complete Section 2 first.'
            return

        proj_dir  = Path(proj_dir_str)
        input_dir = proj_dir / 'input'
        if not input_dir.exists():
            yield '❌ Input directory not found: `{}` — run Section 3 first.'.format(input_dir)
            return

        frames = list(input_dir.glob('*.jpg')) + list(input_dir.glob('*.png'))
        if not frames:
            yield '❌ No images found in `{}` — run Section 3 first.'.format(input_dir)
            return

        _n_frames = len(frames)

        db_path = proj_dir / 'database.db'
        sparse  = proj_dir / 'sparse'
        sparse.mkdir(exist_ok=True)

        for stale in [db_path, db_path.with_suffix('.db-wal'), db_path.with_suffix('.db-shm')]:
            if stale.exists():
                stale.unlink()
                print('[COLMAP] Removed stale file: {}'.format(stale.name))

        if preset != 'Custom':
            _matcher = 'exhaustive' if preset == 'High Quality' else 'sequential'
            overlap  = 15 if preset == 'Balanced' else 10
        else:
            _matcher = matcher

        gpu_flag = '1' if use_gpu_chk else '0'

        if not use_gpu_chk and _n_frames > 80:
            yield (
                '⚠ **CPU SIFT with {} frames ≈ {:.0f} hours** (~90 s/pair × {} pairs).\n\n'
                '**Options:**\n'
                '- Re-run Section 3 with **Quick Preview** (60 frames), then retry\n'
                '- Tick **Use GPU** above (requires Xvfb but is ~50× faster)'
            ).format(_n_frames, _n_frames * 15 * 90 / 3600, _n_frames * 15)
            return

        colmap_env = {**os.environ, 'QT_QPA_PLATFORM': 'offscreen'}

        # ── Xvfb start / restart helper ───────────────────────────────────
        def start_xvfb():
            nonlocal xvfb_proc
            display = ':99'
            subprocess.run(['pkill', '-f', 'Xvfb {}'.format(display)], capture_output=True)
            time.sleep(0.5)
            for lp in ['/tmp/.X99-lock', '/tmp/.X11-unix/X99']:
                try:
                    os.remove(lp)
                except FileNotFoundError:
                    pass
            xvfb_proc = subprocess.Popen(
                ['Xvfb', display, '-screen', '0', '1024x768x24',
                 '-nolisten', 'tcp', '-ac'],
                stdout=subprocess.DEVNULL,
                stderr=subprocess.DEVNULL,
            )
            time.sleep(1.5)
            if xvfb_proc.poll() is not None:
                return False, xvfb_proc.returncode
            print('[COLMAP] Xvfb running on {} (pid={})'.format(display, xvfb_proc.pid))
            colmap_env['DISPLAY'] = display
            return True, 0

        def ensure_xvfb(step_label):
            if not use_gpu_chk:
                return True
            if xvfb_proc is not None and xvfb_proc.poll() is None:
                return True
            print('[COLMAP] Xvfb exited before {} — restarting…'.format(step_label))
            ok, _ = start_xvfb()
            return ok

        if use_gpu_chk:
            yield '⏳ Starting Xvfb virtual display…'
            ok, rc = start_xvfb()
            if not ok:
                yield '❌ **Xvfb failed to start** (exit {}).\nUncheck **Use GPU** to use CPU SIFT.'.format(rc)
                return

        sys.stdout.flush()

        # ── streaming step runner ─────────────────────────────────────────
        def stream_step(label, cmd, timeout_s):
            nonlocal step_log
            step_log = []
            cmd_str  = ' '.join(str(c) for c in cmd)
            print('\n[COLMAP] {}\n[CMD] {}'.format(label, cmd_str))
            sys.stdout.flush()

            try:
                proc = subprocess.Popen(
                    cmd,
                    stdout=subprocess.PIPE,
                    stderr=subprocess.STDOUT,
                    text=True,
                    env=colmap_env,
                )
            except FileNotFoundError as e:
                yield render('❌', label, 'command not found: `{}`'.format(e))
                yield (False, -1)
                return

            deadline     = time.time() + timeout_s
            last_yield_t = time.time()

            for raw_line in proc.stdout:
                line = raw_line.rstrip()
                step_log.append(line)
                all_log.append(line)
                print(line)
                sys.stdout.flush()

                if time.time() > deadline:
                    proc.kill()
                    proc.wait()
                    tmsg = 'timed out after {:.0f} min'.format(timeout_s / 60)
                    print('[COLMAP] TIMEOUT — {}'.format(label))
                    yield render('❌', label, tmsg)
                    yield (False, -1)
                    return

                now = time.time()
                important = any(kw in line.lower() for kw in
                                ('processed file', 'matching image', 'registering',
                                 'error', 'warning', 'failed', 'adding image'))
                if now - last_yield_t >= 2.0 or important:
                    yield render('⏳', label)
                    last_yield_t = now

            proc.wait()
            rc = proc.returncode
            print('[COLMAP] {} exit code: {}'.format(label, rc))
            sys.stdout.flush()

            if rc != 0:
                yield render('❌', label, 'exit code `{}`'.format(rc))
                yield (False, rc)
            else:
                yield render('✅', label, 'done')
                yield (True, 0)

        # ─────────────────────────────────────────────────────────────────
        # Step 1 — Feature extraction
        # ─────────────────────────────────────────────────────────────────
        ok = False
        rc = -1
        for item in stream_step('Feature extraction', [
            'colmap', 'feature_extractor',
            '--database_path',             str(db_path),
            '--image_path',                str(input_dir),
            '--ImageReader.single_camera', '1',
            '--SiftExtraction.use_gpu',    gpu_flag,
        ], TIMEOUT_EXTRACT):
            if isinstance(item, tuple):
                ok, rc = item
            else:
                yield item
        if not ok:
            return

        step_log = []

        if not ensure_xvfb('Sequential matching'):
            yield '❌ Could not restart Xvfb before matching.  Uncheck **Use GPU** and retry.'
            return
        yield render('⏳', 'Starting {} matching…'.format(_matcher))

        # ─────────────────────────────────────────────────────────────────
        # Step 2 — Matching
        # ─────────────────────────────────────────────────────────────────
        if _matcher == 'sequential':
            match_cmd = [
                'colmap', 'sequential_matcher',
                '--database_path',              str(db_path),
                '--SiftMatching.use_gpu',        gpu_flag,
                '--SequentialMatching.overlap',  str(int(overlap)),
            ]
        else:
            match_cmd = [
                'colmap', 'exhaustive_matcher',
                '--database_path',        str(db_path),
                '--SiftMatching.use_gpu', gpu_flag,
            ]

        ok = False
        for item in stream_step('{} matching'.format(_matcher.capitalize()),
                                 match_cmd, TIMEOUT_MATCH):
            if isinstance(item, tuple):
                ok, rc = item
            else:
                yield item
        if not ok:
            return

        step_log = []

        if not ensure_xvfb('Mapper'):
            yield '❌ Could not restart Xvfb before mapper.  Uncheck **Use GPU** and retry.'
            return

        # ─────────────────────────────────────────────────────────────────
        # Step 3 — Mapper
        # ─────────────────────────────────────────────────────────────────
        ok = False
        for item in stream_step('Mapper (sparse reconstruction)', [
            'colmap', 'mapper',
            '--database_path', str(db_path),
            '--image_path',    str(input_dir),
            '--output_path',   str(sparse),
        ], TIMEOUT_MAPPER):
            if isinstance(item, tuple):
                ok, rc = item
            else:
                yield item
        if not ok:
            return

        step_log = []

        if not ensure_xvfb('Image undistortion'):
            yield '❌ Could not restart Xvfb before undistortion.  Uncheck **Use GPU** and retry.'
            return

        # ─────────────────────────────────────────────────────────────────
        # Step 4 — Image undistortion
        # Converts distorted camera model → PINHOLE, required by train.py.
        # image_undistorter --output_type COLMAP writes sparse files flat
        # into dense/sparse/; train.py expects dense/sparse/0/ — so we
        # move the files into a 0/ subdirectory after undistortion.
        # ─────────────────────────────────────────────────────────────────
        dense_dir = proj_dir / 'dense'
        dense_dir.mkdir(exist_ok=True)

        ok = False
        for item in stream_step('Image undistortion', [
            'colmap', 'image_undistorter',
            '--image_path',  str(input_dir),
            '--input_path',  str(sparse / '0'),
            '--output_path', str(dense_dir),
            '--output_type', 'COLMAP',
        ], TIMEOUT_MAPPER):
            if isinstance(item, tuple):
                ok, rc = item
            else:
                yield item
        if not ok:
            return

        # Move flat sparse/ files into sparse/0/ to match train.py's expected layout
        sparse_flat = dense_dir / 'sparse'
        sparse_sub  = sparse_flat / '0'
        sparse_sub.mkdir(exist_ok=True)
        moved = []
        for f in list(sparse_flat.iterdir()):
            if f.is_file():
                f.rename(sparse_sub / f.name)
                moved.append(f.name)
        if moved:
            print('[COLMAP] Moved sparse files to sparse/0/: {}'.format(', '.join(moved)))

        # Point state to the undistorted dataset; train.py reads images from
        # dense/images/ and sparse from dense/sparse/0/ automatically.
        _state['project_dir'] = str(dense_dir)
        print('[COLMAP] project_dir updated to undistorted dataset: {}'.format(dense_dir))

        # ─────────────────────────────────────────────────────────────────
        # Validate output
        # ─────────────────────────────────────────────────────────────────
        sparse_0   = sparse_sub   # dense/sparse/0/
        registered = count_registered_images(sparse_0)
        rate       = registered / _n_frames if _n_frames else 0.0
        _state['sparse_dir'] = str(sparse_0)

        if rate < 0.5:
            verdict = '⚠ Low registration: {}/{} ({:.0f}%) — try exhaustive matcher or more frames.'.format(
                registered, _n_frames, rate * 100)
        elif rate < 0.8:
            verdict = '⚠ Moderate: {}/{} ({:.0f}%)'.format(registered, _n_frames, rate * 100)
        else:
            verdict = '✅ COLMAP complete — {}/{} images registered ({:.0f}%)'.format(
                registered, _n_frames, rate * 100)

        print('[COLMAP] {}'.format(verdict))
        yield (
            '**COLMAP finished** in {elapsed}\n\n---\n\n### {verdict}\n\n'
            '**Full log** (last 40 lines):\n```\n{log}\n```'
        ).format(elapsed=elapsed(), verdict=verdict, log='\n'.join(all_log[-40:]))

    except Exception:
        tb = traceback.format_exc()
        print('[run_colmap FATAL]\n{}'.format(tb))
        sys.stdout.flush()
        yield '### ❌ Fatal error\n```\n{}\n```'.format(tb)

    finally:
        if xvfb_proc is not None:
            try:
                xvfb_proc.terminate()
                xvfb_proc.wait()
            except Exception:
                pass
            print('[COLMAP] Xvfb stopped.')


with gr.Blocks(title='Vid2Splat — COLMAP') as colmap_ui:
    gr.Markdown(
        '### 🗂 COLMAP Structure-from-Motion\n'
        '> Output updates **live** every ~2 seconds. '
        'Raw logs also appear in the **Colab cell output** below.\n\n'
        '> Timeouts: extraction 60 min · matching **120 min** · mapper 60 min · undistortion 60 min'
    )
    with gr.Row():
        c_preset  = gr.Dropdown(['Quick Preview', 'Balanced', 'High Quality', 'Custom'],
                                 label='Preset', value='Balanced')
        c_matcher = gr.Dropdown(['sequential', 'exhaustive'], label='Matcher (Custom)', value='sequential')
        c_overlap = gr.Slider(5, 30, value=15, step=5, label='Sequential overlap (Custom)')
        c_gpu     = gr.Checkbox(label='Use GPU (CUDA SIFT — requires Xvfb)', value=False)
    colmap_btn = gr.Button('Run COLMAP', variant='primary')
    c_status   = gr.Markdown(value='*Press "Run COLMAP" to start.*')
    colmap_btn.click(run_colmap,
                     inputs=[c_preset, c_matcher, c_overlap, c_gpu],
                     outputs=[c_status])

colmap_ui.launch(debug=False, share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Matching image [2/60] in 1.116s
* Running on public URL: https://d4ee5a9906801d3069.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


## Section 5 — 3D Gaussian Splatting Training

In [9]:

# ── 5. Gaussian Splatting Training ────────────────────────────────────────
import gradio as gr
import subprocess, threading, re, time, sys, traceback
from pathlib import Path

GS_REPO = Path('/content/gaussian-splatting')
PRESET_ITERS = {'Quick Preview': 7000, 'Balanced': 15000, 'High Quality': 30000}
PRESET_RES   = {'Quick Preview': 4,    'Balanced': 2,     'High Quality': 1}

training_proc = None
training_log  = []


def start_training(preset, iterations, resolution, adv_visible,
                   dens_until, dens_interval, lr_init, lr_final,
                   opacity_lr, scaling_lr, lambda_dssim):
    global training_proc, training_log

    try:
        _state = sys.modules['__main__'].__dict__.get('state') or globals().get('state')
        if not _state:
            return '❌ `state` not found — re-run Section 2 first.', gr.update()

        # project_dir is updated to dense/ after undistortion in Section 4
        proj_dir = Path(_state.get('project_dir', '/content/vid2splat/scene'))
        output   = proj_dir.parent / (proj_dir.name + '_output')
        output.mkdir(parents=True, exist_ok=True)

        # After undistortion + move: sparse lives at proj_dir/sparse/0/
        sparse_0 = proj_dir / 'sparse' / '0'
        if not sparse_0.exists():
            return (
                '⚠ Run Section 4 (COLMAP) first — `{}` not found.'.format(sparse_0),
                gr.update()
            )

        if preset != 'Custom':
            iterations = PRESET_ITERS[preset]
            resolution = PRESET_RES[preset]
            dens_until = int(iterations * 0.8)

        # sh_degree is ALWAYS 3 — required by the Unity VR Viewer
        sh_degree = 3

        cmd = [
            'python', str(GS_REPO / 'train.py'),
            '--source_path',            str(proj_dir),
            '--model_path',             str(output),
            '--iterations',             str(int(iterations)),
            '--sh_degree',              str(sh_degree),
            '--resolution',             str(int(resolution)),
            '--densify_until_iter',     str(int(dens_until)),
            '--densification_interval', str(int(dens_interval)),
            '--position_lr_init',       str(lr_init),
            '--position_lr_final',      str(lr_final),
            '--opacity_lr',             str(opacity_lr),
            '--scaling_lr',             str(scaling_lr),
            '--lambda_dssim',           str(lambda_dssim),
        ]

        training_log = []
        _state['output_dir'] = str(output)
        _state['training_iterations'] = int(iterations)

        print(f'[TRAIN] source_path: {proj_dir}')
        print(f'[TRAIN] sparse check: {sparse_0}  exists={sparse_0.exists()}')
        print(f'[TRAIN] cmd: {" ".join(cmd)}')
        sys.stdout.flush()

        training_proc = subprocess.Popen(
            cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

        def stream():
            for line in training_proc.stdout:
                training_log.append(line.rstrip())
            training_proc.wait()
            print(f'[TRAIN] process exited with code {training_proc.returncode}')

        threading.Thread(target=stream, daemon=True).start()
        return f'🚀 Training started ({int(iterations):,} iters, sh_degree=3, resolution=1/{int(resolution)})', gr.update()

    except Exception:
        tb = traceback.format_exc()
        print(f'[TRAIN ERROR]\n{tb}')
        return f'❌ Training failed to start:\n```\n{tb}\n```', gr.update()


def poll_training():
    global training_proc, training_log

    _state = sys.modules['__main__'].__dict__.get('state') or globals().get('state') or {}

    if not training_log:
        if training_proc is None:
            return '⏳ Press "Start training" first.', '', ''
        return '⏳ Waiting for first output...', '', ''

    recent = '\n'.join(training_log[-30:])

    # Parse last iteration and loss
    last_iter = 0
    last_loss = 'n/a'
    for line in reversed(training_log):
        m = re.search(r'iteration\s+(\d+)', line, re.I)
        if m:
            last_iter = int(m.group(1))
        m2 = re.search(r'loss[\s:=]+([0-9.eE+\-]+)', line, re.I)
        if m2:
            last_loss = m2.group(1)
        if last_iter:
            break

    total_iters = _state.get('training_iterations', 15000)
    pct = '{:.1f}%'.format(last_iter / total_iters * 100) if total_iters else ''
    done = training_proc is not None and training_proc.poll() is not None
    prefix = '✅ Done' if done else '⏳ Training'
    status = '{} — iter {:,}/{:,} ({}) | loss={}'.format(prefix, last_iter, total_iters, pct, last_loss)

    return status, recent, ''


with gr.Blocks(title='Vid2Splat — Training') as train_ui:
    gr.Markdown('### 🧠 3D Gaussian Splatting Training\n'
                '> `sh_degree` is always **3** — hardcoded for Unity VR Viewer compatibility.')
    with gr.Row():
        t_preset = gr.Dropdown(['Quick Preview', 'Balanced', 'High Quality', 'Custom'],
                                label='Preset', value='Balanced')
        t_iters  = gr.Slider(1000, 30000, value=15000, step=1000, label='Iterations (Custom)')
        t_res    = gr.Dropdown([1, 2, 4], label='Resolution divisor (Custom)', value=2)

    with gr.Accordion('Advanced hyperparameters', open=False):
        adv_chk     = gr.Checkbox(label='Use custom values below', value=False)
        t_dens_util = gr.Slider(1000, 30000, value=12000, step=500, label='densify_until_iter')
        t_dens_int  = gr.Slider(50, 500, value=100, step=50, label='densification_interval')
        t_lr_init   = gr.Number(value=0.00016, label='position_lr_init')
        t_lr_final  = gr.Number(value=0.0000016, label='position_lr_final')
        t_op_lr     = gr.Number(value=0.05, label='opacity_lr')
        t_sc_lr     = gr.Number(value=0.005, label='scaling_lr')
        t_dssim     = gr.Slider(0.0, 1.0, value=0.2, step=0.05, label='lambda_dssim')

    train_btn  = gr.Button('Start training', variant='primary')
    poll_btn   = gr.Button('Refresh progress')
    t_status   = gr.Markdown()
    t_log      = gr.Textbox(label='Training log (last 30 lines)', lines=15, interactive=False)
    t_dummy    = gr.Textbox(visible=False)

    train_btn.click(start_training,
                    inputs=[t_preset, t_iters, t_res, adv_chk,
                            t_dens_util, t_dens_int, t_lr_init, t_lr_final,
                            t_op_lr, t_sc_lr, t_dssim],
                    outputs=[t_status, t_dummy])
    poll_btn.click(poll_training, outputs=[t_status, t_log, t_dummy])

train_ui.launch(debug=False, share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://a8702f37cfd432aa72.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


## Section 6 — Results, Validation & Export

In [10]:
# ── 6. Results, validation, download & preview ───────────────────────────
import gradio as gr
import struct, zipfile, os, shutil
import plotly.graph_objects as go
import numpy as np
from pathlib import Path

REQUIRED_FIELDS = (
    ['x','y','z','nx','ny','nz','f_dc_0','f_dc_1','f_dc_2'] +
    [f'f_rest_{i}' for i in range(45)] +
    ['opacity','scale_0','scale_1','scale_2','rot_0','rot_1','rot_2','rot_3']
)  # 62 fields


def find_ply():
    out = state.get('output_dir')
    iters = state.get('training_iterations', 30000)
    if not out:
        return None
    exact = Path(out) / 'point_cloud' / f'iteration_{iters}' / 'point_cloud.ply'
    if exact.exists():
        return exact
    pc_root = Path(out) / 'point_cloud'
    if pc_root.exists():
        candidates = sorted(pc_root.glob('iteration_*/point_cloud.ply'))
        if candidates:
            return candidates[-1]
    return None


def validate_ply_file(ply_path):
    if not ply_path or not Path(ply_path).exists():
        return {}
    fields, n_gauss = [], 0
    is_ble = False
    with open(ply_path, 'rb') as f:
        for _ in range(200):
            line = f.readline().decode('ascii','replace').strip()
            if 'binary_little_endian' in line:
                is_ble = True
            if line.startswith('element vertex'):
                try:
                    n_gauss = int(line.split()[-1])
                except ValueError:
                    pass
            if line.startswith('property float'):
                fields.append(line.split()[-1])
            if line == 'end_header':
                break
    missing = [f for f in REQUIRED_FIELDS if f not in fields]
    size_mb = Path(ply_path).stat().st_size / 1024**2
    return dict(n_gaussians=n_gauss, size_mb=size_mb,
                is_ble=is_ble, missing=missing,
                passed=(n_gauss > 0 and is_ble and not missing and size_mb > 0.5))


def build_report():
    ply = find_ply()
    if not ply:
        return '⚠ No PLY found — run training first.', None, None

    v = validate_ply_file(str(ply))
    if v.get('passed'):
        status = (f'✅ **PLY validation passed**\n'
                  f'- Gaussians: {v["n_gaussians"]:,}\n'
                  f'- File size: {v["size_mb"]:.1f} MB\n'
                  f'- Format: binary_little_endian ✓\n'
                  f'- All 62 vertex fields present ✓')
    else:
        issues = []
        if not v.get('is_ble'): issues.append('Not binary_little_endian')
        if v.get('missing'):    issues.append(f'Missing fields: {v["missing"]}')
        if v.get('size_mb',0) < 0.5: issues.append('File too small')
        status = '⚠ PLY issues:\n' + '\n'.join(f'- {i}' for i in issues)

    # ── Quick scatter preview of Gaussian centres ──────────────────────────
    try:
        from plyfile import PlyData
        pdata = PlyData.read(str(ply))
        verts = pdata['vertex']
        n = min(10000, len(verts))
        idx = np.random.choice(len(verts), n, replace=False)
        xs = np.array(verts['x'][idx], dtype=float)
        ys = np.array(verts['y'][idx], dtype=float)
        zs = np.array(verts['z'][idx], dtype=float)
        opacities = 1 / (1 + np.exp(-np.array(verts['opacity'][idx], dtype=float)))
        fig = go.Figure(data=[go.Scatter3d(
            x=xs, y=ys, z=zs,
            mode='markers',
            marker=dict(size=1.5,
                        color=opacities,
                        colorscale='Viridis',
                        opacity=0.6),
        )])
        fig.update_layout(title=f'{n:,} Gaussians (sample preview)',
                          scene=dict(aspectmode='data'),
                          margin=dict(l=0,r=0,t=30,b=0))
    except Exception as e:
        fig = None
        print('Preview skipped:', e)

    # ── Zip for download ───────────────────────────────────────────────────
    zip_path = Path(state.get('output_dir', '/content')) / 'point_cloud.zip'
    with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zf:
        zf.write(ply, 'point_cloud.ply')
    state['zip_path'] = str(zip_path)

    return status, fig, str(zip_path)


with gr.Blocks(title='Vid2Splat — Results') as results_ui:
    gr.Markdown('### 📦 Results, Validation & Export')
    report_btn  = gr.Button('Generate report & preview', variant='primary')
    r_status    = gr.Markdown()
    r_plot      = gr.Plot(label='3D Gaussian preview (sample)')
    r_download  = gr.File(label='Download point_cloud.zip')
    report_btn.click(build_report, outputs=[r_status, r_plot, r_download])

    gr.Markdown('''
---
### 🎮 Loading in the Unity VR Viewer
1. Copy `point_cloud.ply` (from the zip) to the Unity project folder.
2. Open the Unity project in the **Unity Editor** on an **NVIDIA GPU machine**.
3. Set `GaussianSplattingModel → Model File Path` to the `.ply` file path.
4. Press **Play**.

> The `.ply` uses `sh_degree=3` and binary-little-endian format — fully compatible with the `GaussianSplattingPlugin`.
    ''')

results_ui.launch(debug=False, share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://e0c13b9e5dca8c3bb6.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


## (Optional) Save to Google Drive

In [ ]:
# ── 7. (Optional) Save PLY to Google Drive ───────────────────────────────
SAVE_TO_DRIVE = False   # ← set True to enable
DRIVE_DEST    = 'MyDrive/Vid2Splat/'

if SAVE_TO_DRIVE:
    from google.colab import drive
    import shutil
    from pathlib import Path

    drive.mount('/content/drive')
    dest_dir = Path('/content/drive') / DRIVE_DEST
    dest_dir.mkdir(parents=True, exist_ok=True)

    ply_zip = state.get('zip_path')
    if ply_zip and Path(ply_zip).exists():
        dst = dest_dir / Path(ply_zip).name
        shutil.copy2(ply_zip, dst)
        print(f'✅ Saved to Drive: {dst}')
    else:
        print('⚠ No zip found — run Section 6 first.')
else:
    print('ℹ SAVE_TO_DRIVE is False — skipping.')

ℹ SAVE_TO_DRIVE is False — skipping.


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
